### Section 4


This file runs the experiments and returns the top performing models as a dictionary of hyper-parameters. It also creates all figures in the paper (except for those specific to the causal PDP and CD diagrams).